In [8]:
from window_operate import *
from gaming_func import *
from notice_func import *
import sys,time,os
import threading

from ctypes import *
from ctypes.wintypes import *

def get_current_size(hwnd):
    try:
        f = ctypes.windll.dwmapi.DwmGetWindowAttribute
    except WindowsError:
        f = None
    if f:
        rect = ctypes.wintypes.RECT()
        DWMWA_EXTENDED_FRAME_BOUNDS = 9
        f(ctypes.wintypes.HWND(hwnd),
          ctypes.wintypes.DWORD(DWMWA_EXTENDED_FRAME_BOUNDS),
          ctypes.byref(rect),
          ctypes.sizeof(rect)
          )
        size = (rect.right - rect.left, rect.bottom - rect.top)        
        return size

hwnd = get_window_handle(title="咸鱼之王")
if hwnd == 0:
    print("游戏未启动，退出...")
    time.sleep(3)
    os._exit(1)
print_help()
print("Start...tap help for more details.")
w1,h1=get_current_size(hwnd)
rect = win32gui.GetWindowRect(hwnd)
x,y=rect[0],rect[1]
print(x,y)
w,h=rect[2] - x,rect[3] - y
print(w1,h1)
print(w,h)

-----------------
1.       显示窗口
2.       隐藏窗口
shoucai. 收离线奖励
tower.   咸将塔
help.    帮助
exit.    退出
-----------------
Start...tap help for more details.
978 138
905 1674
603 1116


In [2]:
import win32gui
hwnd_title = dict() 
def get_all_hwnd(hwnd,mouse):
    if win32gui.IsWindow(hwnd) and win32gui.IsWindowEnabled(hwnd) and win32gui.IsWindowVisible(hwnd):
        hwnd_title.update({hwnd:win32gui.GetWindowText(hwnd)})
win32gui.EnumWindows(get_all_hwnd, 0)

for h,t in hwnd_title.items():
    if t is not "":
        print(([h], [t]))

([67090], ['● test.ipynb - SaltedFishBot - Visual Studio Code'])
([132226], ['Amovement/SaltedFishBot: 咸鱼之王pc脚本 - Google Chrome'])
([67304], ['设置'])
([67738], ['新建 文本文档 - Notepad'])
([132638], ['安装'])
([66624], ['NVIDIA GeForce Overlay'])
([131122], ['Windows 输入体验'])
([66972], ['Clash for Windows'])
([66700], ['梦回2018年！假面骑士时王全形态变身合集_哔哩哔哩_bilibili - Google Chrome'])
([329632], ['咸鱼之王'])
([590050], ['图片查看'])
([263984], ['微信'])
([65920], ['Program Manager'])


<>:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\ONE\AppData\Local\Temp\ipykernel_11660\1463654781.py:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if t is not "":


In [4]:
from window_operate import *
from gaming_func import *
from notice_func import *
import sys,time,os
import threading

from ctypes import *
from ctypes.wintypes import *

def get_current_size(hwnd):
    try:
        f = ctypes.windll.dwmapi.DwmGetWindowAttribute
    except WindowsError:
        f = None
    if f:
        rect = ctypes.wintypes.RECT()
        DWMWA_EXTENDED_FRAME_BOUNDS = 9
        f(ctypes.wintypes.HWND(hwnd),
          ctypes.wintypes.DWORD(DWMWA_EXTENDED_FRAME_BOUNDS),
          ctypes.byref(rect),
          ctypes.sizeof(rect)
          )
        size = (rect.right - rect.left, rect.bottom - rect.top)        
        return size

hwnd = get_window_handle(title="咸鱼之王")
if hwnd == 0:
    print("游戏未启动，退出...")
    time.sleep(3)
    os._exit(1)
print_help()
print("Start...tap help for more details.")
w1,h1=get_current_size(hwnd)
rect = win32gui.GetWindowRect(hwnd)
x,y=rect[0],rect[1]
print(x,y)
w,h=rect[2] - x,rect[3] - y
print(w1,h1)
print(w,h)

-----------------
1.       显示窗口
2.       隐藏窗口
shoucai. 收离线奖励
tower.   咸将塔
help.    帮助
exit.    退出
-----------------
Start...tap help for more details.
24 9
675 1266
450 844


In [ ]:
x,y=rect[0],rect[1]
w,h=rect[2] - x,rect[3] - y

In [1]:
from ctypes import windll, byref, c_ubyte
from ctypes.wintypes import RECT, HWND
import numpy as np
import cv2
import win32con
import win32gui

GetDC = windll.user32.GetDC
CreateCompatibleDC = windll.gdi32.CreateCompatibleDC
GetClientRect = windll.user32.GetClientRect
CreateCompatibleBitmap = windll.gdi32.CreateCompatibleBitmap
SelectObject = windll.gdi32.SelectObject
BitBlt = windll.gdi32.BitBlt
GetBitmapBits = windll.gdi32.GetBitmapBits
DeleteObject = windll.gdi32.DeleteObject
ReleaseDC = windll.user32.ReleaseDC

# 排除缩放干扰
# windll.user32.SetProcessDPIAware()

def capture(handle: HWND):
    """窗口客户区截图

    Args:
        handle (HWND): 要截图的窗口句柄

    Returns:
        numpy.ndarray: 截图数据
    """
    # 获取窗口客户区的大小
    r = RECT()
    GetClientRect(handle, byref(r))
    width, height = r.right, r.bottom
    # 开始截图
    dc = GetDC(handle)
    cdc = CreateCompatibleDC(dc)
    bitmap = CreateCompatibleBitmap(dc, width, height)
    SelectObject(cdc, bitmap)
    BitBlt(cdc, 0, 0, width, height, dc, 0, 0, win32con.SRCCOPY)
    # 截图是BGRA排列，因此总元素个数需要乘以4
    total_bytes = width*height*4
    buffer = bytearray(total_bytes)
    byte_array = c_ubyte*total_bytes
    GetBitmapBits(bitmap, total_bytes, byte_array.from_buffer(buffer))
    DeleteObject(bitmap)
    DeleteObject(cdc)
    ReleaseDC(handle, dc)
    image =np.frombuffer(buffer, dtype=np.uint8).reshape(height, width, 4)
    cv2.imshow("Capture Test", image)
    cv2.waitKey()


